In [2]:
import tkinter as tk
import random

# Game settings
screen_width = 600
screen_height = 600
block_size = 20
initial_speed = 100  # Initial game speed in milliseconds
speed_increment = 5  # Speed increase per food eaten
obstacle_count = 5  # Number of obstacles

# Initialize the snake and game state
snake = [(100, 100), (80, 100), (60, 100)]  # Starting coordinates of the snake (initially 3 blocks long)
snake_direction = "Right"  # Direction in which the snake is moving
food_position = (0, 0)  # Initial food position
score = 0
game_over = False
obstacles = []
game_paused = False

# Create the game window
root = tk.Tk()
root.title("Difficult Snake Game")
root.geometry(f"{screen_width}x{screen_height}")
root.config(bg="black")

# Create the canvas for the game
canvas = tk.Canvas(root, width=screen_width, height=screen_height, bg="white")
canvas.pack()

# Function to draw the snake, food, and obstacles
def draw_game():
    global food_position, obstacles
    canvas.delete("all")

    # Draw the snake
    for segment in snake:
        canvas.create_rectangle(segment[0], segment[1], segment[0] + block_size, segment[1] + block_size, fill="green")

    # Draw the food
    canvas.create_rectangle(food_position[0], food_position[1], food_position[0] + block_size, food_position[1] + block_size, fill="red")

    # Draw obstacles
    for obstacle in obstacles:
        canvas.create_rectangle(obstacle[0], obstacle[1], obstacle[0] + block_size, obstacle[1] + block_size, fill="black")

    # Draw the score
    canvas.create_text(10, 10, anchor="nw", text=f"Score: {score}", font=("Arial", 16, "bold"))

# Function to generate new food
def generate_food():
    global food_position
    food_position = (random.randint(0, (screen_width - block_size) // block_size) * block_size,
                     random.randint(0, (screen_height - block_size) // block_size) * block_size)

# Function to generate obstacles
def generate_obstacles():
    global obstacles
    obstacles = []
    for _ in range(obstacle_count):
        obstacles.append((
            random.randint(0, (screen_width - block_size) // block_size) * block_size,
            random.randint(0, (screen_height - block_size) // block_size) * block_size
        ))

# Function to move the snake
def move_snake():
    global snake, snake_direction, food_position, score, game_over, obstacles, game_paused

    if game_over or game_paused:
        return

    head_x, head_y = snake[0]

    # Determine new head position based on current direction
    if snake_direction == "Right":
        head_x += block_size
    elif snake_direction == "Left":
        head_x -= block_size
    elif snake_direction == "Up":
        head_y -= block_size
    elif snake_direction == "Down":
        head_y += block_size

    # Wrap around walls
    if head_x < 0:
        head_x = screen_width - block_size
    elif head_x >= screen_width:
        head_x = 0
    if head_y < 0:
        head_y = screen_height - block_size
    elif head_y >= screen_height:
        head_y = 0

    # New head position
    new_head = (head_x, head_y)

    # Check for collisions with itself
    if new_head in snake:
        game_over = True
        canvas.create_text(screen_width // 2, screen_height // 2, text="Game Over!", font=("Arial", 30, "bold"), fill="red")
        return

    # Check for collisions with obstacles
    if new_head in obstacles:
        game_over = True
        canvas.create_text(screen_width // 2, screen_height // 2, text="Game Over!", font=("Arial", 30, "bold"), fill="red")
        return

    # Add the new head to the snake
    snake = [new_head] + snake

    # Check if snake eats food
    if new_head == food_position:
        score += 1
        generate_food()
        generate_obstacles()
    else:
        snake.pop()  # Remove the tail

    draw_game()

    # Increase speed after each food is eaten
    global speed_increment
    root.after(initial_speed - score * speed_increment, move_snake)

# Function to change the direction of the snake
def change_direction(event):
    global snake_direction

    if event.keysym == "Right" and snake_direction != "Left":
        snake_direction = "Right"
    elif event.keysym == "Left" and snake_direction != "Right":
        snake_direction = "Left"
    elif event.keysym == "Up" and snake_direction != "Down":
        snake_direction = "Up"
    elif event.keysym == "Down" and snake_direction != "Up":
        snake_direction = "Down"

# Function to start the game
def start_game():
    global score, snake, game_over, game_paused
    score = 0
    snake = [(100, 100), (80, 100), (60, 100)]
    game_over = False
    game_paused = False
    generate_food()
    generate_obstacles()
    move_snake()

# Function to pause the game
def pause_game():
    global game_paused
    if not game_over:
        game_paused = True
        canvas.create_text(screen_width // 2, screen_height // 2, text="Game Paused", font=("Arial", 30, "bold"), fill="blue")

# Function to resume the game
def resume_game():
    global game_paused
    if not game_over:
        game_paused = False
        move_snake()

# Function to restart the game after game over
def restart_game():
    global game_over
    if game_over:
        start_game()

# Function to create new game button after game over
def new_game():
    global game_over
    if game_over:
        start_game()

# Create buttons for the game controls
start_button = tk.Button(root, text="Start Game", command=start_game, width=15, height=2, bg="green", font=("Arial", 14, "bold"))
start_button.pack(side="top", pady=5)

pause_button = tk.Button(root, text="Pause Game", command=pause_game, width=15, height=2, bg="yellow", font=("Arial", 14, "bold"))
pause_button.pack(side="top", pady=5)

resume_button = tk.Button(root, text="Resume Game", command=resume_game, width=15, height=2, bg="blue", font=("Arial", 14, "bold"))
resume_button.pack(side="top", pady=5)

restart_button = tk.Button(root, text="Restart Game", command=restart_game, width=15, height=2, bg="red", font=("Arial", 14, "bold"))
restart_button.pack(side="top", pady=5)

new_game_button = tk.Button(root, text="New Game", command=new_game, width=15, height=2, bg="purple", font=("Arial", 14, "bold"))
new_game_button.pack(side="top", pady=5)

# Bind keyboard events
root.bind("<Right>", change_direction)
root.bind("<Left>", change_direction)
root.bind("<Up>", change_direction)
root.bind("<Down>", change_direction)

# Start the game loop
start_game()
root.mainloop()
